## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/dangkhoa/spark-2.3.1-bin-hadoop2.7')

In [2]:
from IPython.display import display

## Session

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic_Regression').getOrCreate()

## Load dataset

In [4]:
raw_data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

display(raw_data.printSchema())
display(raw_data.show(5))
display(raw_data.describe().show())

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



None

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

None

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

None

## Vector Assembler

In [5]:
from pyspark.ml.feature import VectorAssembler

# Define assembler 
assembler = VectorAssembler(
    inputCols=[
        'Age',
        'Total_Purchase',
        'Account_Manager',
        'Years',
        'Num_Sites'],
    outputCol='features')

# transform
vector_indexed_data = assembler.transform(raw_data)

# Extract Final data
final_data = vector_indexed_data.select('features','churn')

display(final_data.printSchema())
display(final_data.show(5))

root
 |-- features: vector (nullable = true)
 |-- churn: integer (nullable = true)



None

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
+--------------------+-----+
only showing top 5 rows



None

## Split Train/Test set

In [6]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

## Model

In [7]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='churn')

In [8]:
# Train
model = lr.fit(train_data)

In [9]:
# Stats
model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                639|                639|
|   mean| 0.1705790297339593|0.12832550860719874|
| stddev|0.37643536323564897|0.33471390556752484|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



## Evaluate

In [10]:
pred = model.evaluate(test_data)

pred.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,8670.98,0.0...|    0|[7.81087400441976...|[0.99959486054871...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.17219021958301...|[0.76354068001459...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.34254233724621...|[0.98716349155391...|       0.0|
|[29.0,13255.05,1....|    0|[4.11737820016009...|[0.98397386012021...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.13760643004873...|[0.98428974196722...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluation = BinaryClassificationEvaluator(
    rawPredictionCol='prediction',
    labelCol='churn')

auc = evaluation.evaluate(pred.predictions)

auc

0.7455654101995566